<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/049_AgentRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Agent.run()` is the **core orchestration loop** that turns your tools, goals, and environment into a functioning, autonomous agent.

---

## 🧠 What Is `Agent.run()`?

Think of `Agent.run(user_input)` as the **main driver** of the agent's behavior. It:

1. Accepts user input (e.g., “Write a README”)
2. Constructs a prompt using all relevant context (goals, tools, memory)
3. Sends that prompt to the LLM
4. Parses the LLM’s response to identify which action/tool to run
5. Executes the tool with arguments from the LLM
6. Stores the result in memory
7. Repeats until a **terminal** action is selected (like `terminate()`)

---

## 🧱 Typical Structure of `Agent.run()`

Here’s a conceptual breakdown of what `run()` might look like:

```python
def run(self, user_input: str, max_iterations: int = 10):
    self.memory.add_user_input(user_input)

    for _ in range(max_iterations):
        # Step 1: Build prompt using AgentLanguage
        prompt = self.agent_language.construct_prompt(
            goals=self.goals,
            actions=self.action_registry.get_actions(),
            memory=self.memory,
            environment=self.environment
        )

        # Step 2: Send prompt to LLM and get a response
        response = self.generate_response(prompt)

        # Step 3: Parse the tool/function call
        invocation = self.agent_language.parse_response(response)
        action = self.action_registry.get_action(invocation["tool"])

        # Step 4: Execute the tool/function
        result = self.environment.execute_action(action, invocation["args"])

        # Step 5: Update memory
        self.memory.add_memory({"type": "assistant", "content": response})
        self.memory.add_memory({"type": "user", "content": json.dumps(result)})

        # Step 6: Check termination
        if action.terminal:
            break

    return self.memory
```

---

## 🔍 What You'll See in the Real Implementation

The real `Agent.run()` might be more modular, but it **still follows this loop**. Key pieces:

| Step                | Component Involved                 |
| ------------------- | ---------------------------------- |
| Prompt construction | `AgentLanguage.construct_prompt()` |
| LLM call            | `generate_response(prompt)`        |
| Response parsing    | `AgentLanguage.parse_response()`   |
| Action lookup       | `ActionRegistry.get_action(name)`  |
| Action execution    | `Environment.execute_action()`     |
| Memory storage      | `Memory.add_memory()`              |
| Loop termination    | `action.terminal`                  |

---

## 📦 What Gets Passed into the Loop?

* `goals`: what the agent is trying to achieve
* `action_registry`: tools it can use
* `memory`: all prior steps and tool results
* `generate_response`: function that calls your LLM
* `agent_language`: parser and prompt constructor
* `environment`: how tools actually get run

---

## ✅ What You Can Customize

Want your agent to reason differently? Add logic to `Agent.run()`.
Want your agent to retry on certain errors? Handle that in this loop.
Want to support multiple users or a GUI? This is the entry point.





### 🌀 The Agent Loop Is a Goal-Driven Conversation

Each iteration of `Agent.run()` is like saying:

> “Hey LLM, given what we’re trying to do (**the Goal**) and what we’ve already tried (**the Memory**), what tool should we use next, and how?”

#### ✅ Step-by-Step Recap:

1. **Start with a Goal**:
   *"Generate a README by reading all Python files."*

2. **Construct Prompt**:
   The prompt includes:

   * 📌 Current goals
   * 🧠 Memory (e.g. already read `file1.py`)
   * 🧰 Available actions

3. **LLM Chooses Action**:
   *“Let me call `read_project_file(name='file2.py')`”*

4. **Agent Executes It**:
   The environment runs the tool, gets results.

5. **Memory Is Updated**:
   Both the LLM response and tool output go into memory.

6. **Loop Continues**:
   Now the next prompt includes more memory, so the LLM has **more context**.

7. **Eventually...**
   The LLM says: *“Call `terminate()` with this final README text”*.

---

### 🧠 Why This Works So Well with LLMs

LLMs are **good at reasoning iteratively** — making one decision at a time based on what they know so far. That’s why a structured memory + goal setup unlocks their full power.

* Prompting says: “Solve this giant thing in one message.”
* Agents say: “Let’s think step-by-step until we’re done.”

---

### 🛠️ Your Job as the Agent Designer

You don’t need to script every decision. You **design the loop and the ingredients**:

* Clear GOALS
* Useful ACTIONS
* A reliable MEMORY system
* A structured ENVIRONMENT
* A parsing & prompting system (AgentLanguage)

And then the LLM drives the decisions each time through the loop.






## 🧱 A Minimal Agent: Scaffolding Overview

This is the skeleton of how the agent works — with the core GAME parts in place.

```python
# === 1. Define a Goal ===
goals = [
    Goal(priority=1, name="Summarize Codebase", description="Read project files and generate a README.")
]

# === 2. Define Tools (Actions) ===
def list_files():
    ...

def read_file(name: str):
    ...

def terminate(message: str):
    ...

# === 3. Register Actions ===
action_registry = ActionRegistry()
action_registry.register(Action(name="list_files", function=list_files, ...))
action_registry.register(Action(name="read_file", function=read_file, ...))
action_registry.register(Action(name="terminate", function=terminate, terminal=True, ...))

# === 4. Define the Agent Components ===
agent_language = AgentFunctionCallingActionLanguage()
environment = Environment()
memory = Memory()

# === 5. Create Agent ===
agent = Agent(
    goals=goals,
    action_registry=action_registry,
    agent_language=agent_language,
    generate_response=generate_response,  # You supply this
    environment=environment
)
```

---

## 🔁 6. The Agent Loop (What `agent.run()` Actually Does)

Under the hood, the `agent.run()` call looks like this:

```python
while not terminated:
    # Step 1: Build a prompt using goals, actions, memory
    prompt = agent_language.construct_prompt(goals, memory, actions)

    # Step 2: Send prompt to LLM and get a tool call
    response = generate_response(prompt)

    # Step 3: Parse response -> which action + args?
    action, invocation = agent_language.parse_response(response)

    # Step 4: Run the action in the environment
    result = environment.execute_action(action, invocation["args"])

    # Step 5: Save response + result to memory
    memory.add(response, result)

    # Step 6: Check if we're done
    if action.terminal:
        break
```

---

## ✅ What This Shows You

* The loop is **goal-oriented**, not hardcoded.
* Every iteration is LLM-driven: you don’t specify what happens next.
* The `Agent` just holds the orchestration logic — everything else is modular.
* If you plug in different tools or change the goal, **you get a whole new behavior**.




## 🧪 Example Agent: "Summarize Two Files"

### 🎯 Goal:

Read two files and return a simple summary.

---

### 🔧 Tools (Actions):

We’ll create 3 very basic functions:

* `list_files()` – return two hardcoded file names.
* `read_file(file_name)` – return the content of a dummy file.
* `terminate(message)` – end the loop with the final output.

---

### 🧱 Step-by-step Code (Basic Agent)

```python
from typing import List, Callable
from dataclasses import dataclass

# --- Action and Registry ---
class Action:
    def __init__(self, name, function, description, parameters, terminal=False):
        self.name = name
        self.function = function
        self.description = description
        self.parameters = parameters
        self.terminal = terminal

class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str):
        return self.actions.get(name)

    def get_actions(self):
        return list(self.actions.values())

# --- Goal ---
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

# --- Memory (simple list) ---
class Memory:
    def __init__(self):
        self.memories = []

    def add(self, m):
        self.memories.append(m)

    def get_memories(self):
        return self.memories

# --- Environment ---
class Environment:
    def execute_action(self, action: Action, args: dict):
        return action.function(**args)

# --- LLM Parser & Prompt ---
class DummyAgentLanguage:
    def construct_prompt(self, goals, memory, actions):
        return memory.get_memories()

    def parse_response(self, response):
        # Hardcoded fake logic for simplicity
        step = len(response)
        if step == 0:
            return {"tool": "list_files", "args": {}}
        elif step == 1:
            return {"tool": "read_file", "args": {"file_name": "file1.txt"}}
        elif step == 2:
            return {"tool": "read_file", "args": {"file_name": "file2.txt"}}
        else:
            return {"tool": "terminate", "args": {"message": "file1: intro\nfile2: details"}}

# --- Agent Loop ---
class Agent:
    def __init__(self, goals, agent_language, action_registry, generate_response, environment):
        self.goals = goals
        self.agent_language = agent_language
        self.actions = action_registry
        self.generate_response = generate_response
        self.environment = environment

    def run(self, user_input, max_iterations=5):
        memory = Memory()
        for _ in range(max_iterations):
            prompt = self.agent_language.construct_prompt(self.goals, memory, self.actions)
            response = self.generate_response(prompt)
            action_def = self.actions.get_action(response["tool"])
            result = self.environment.execute_action(action_def, response["args"])
            memory.add({"tool_call": response, "result": result})
            if action_def.terminal:
                break
        return memory

# === Define Basic Actions ===
def list_files():
    return ["file1.txt", "file2.txt"]

def read_file(file_name: str):
    return f"Contents of {file_name}"

def terminate(message: str):
    return message

# === Set up the Agent ===
goals = [Goal(1, "Summarize", "Read files and summarize them.")]

registry = ActionRegistry()
registry.register(Action("list_files", list_files, "", {}, terminal=False))
registry.register(Action("read_file", read_file, "", {
    "type": "object",
    "properties": {"file_name": {"type": "string"}},
    "required": ["file_name"]
}, terminal=False))
registry.register(Action("terminate", terminate, "", {
    "type": "object",
    "properties": {"message": {"type": "string"}},
}, terminal=True))

# Dummy LLM — just a fake LLM decision sequence
def generate_response(prompt):
    return DummyAgentLanguage().parse_response(prompt)

# Run the Agent
agent = Agent(goals, DummyAgentLanguage(), registry, generate_response, Environment())
final_memory = agent.run("Summarize my project")

for m in final_memory.get_memories():
    print(m)
```

---

### 🧠 What You’ll See

You should get output like:

```python
{'tool_call': {'tool': 'list_files', 'args': {}}, 'result': ['file1.txt', 'file2.txt']}
{'tool_call': {'tool': 'read_file', 'args': {'file_name': 'file1.txt'}}, 'result': 'Contents of file1.txt'}
{'tool_call': {'tool': 'read_file', 'args': {'file_name': 'file2.txt'}}, 'result': 'Contents of file2.txt'}
{'tool_call': {'tool': 'terminate', 'args': {'message': 'file1: intro\nfile2: details'}}, 'result': 'file1: intro\nfile2: details'}
```

---

## 🔍 What You Just Built

✅ Agent with loop
✅ Goal-oriented decision-making
✅ Tool calls with memory
✅ Termination based on task completion
✅ LLM "thinking" handled by hardcoded decisions (you can swap this with real LLM calls later)






## 🌍 What is the `Environment`?

In the GAME agent architecture, the **`Environment`** is the component responsible for **actually executing the agent’s chosen actions**.

It’s the bridge between:

* The **LLM’s decisions** (e.g., “I want to read a file named `foo.py`”)
* And the **real world effects** (actually opening and reading that file).

---

## 🧠 Analogy: Agent = Brain, Environment = Hands

* The **Agent** decides *what* to do → “I want to call `read_file('file1.txt')`”.
* The **Environment** does the work → actually opens `file1.txt` and returns the content.

You can think of the environment as the **sandbox or context** the agent operates inside.

---

## ✅ What Does the `Environment` Usually Do?

It handles all the *side effects* of running the agent:

* Reading or writing files
* Making API requests
* Querying a database
* Sending an email
* Running a command line instruction
* Saving outputs, logging results
* Calling third-party libraries

---

### 🧱 Minimal Example

```python
class Environment:
    def execute_action(self, action: Action, args: dict):
        return action.function(**args)
```

In this simple case:

* The environment takes the `action.function`
* Applies the `args` the LLM provided
* Returns the result

But you could easily expand this to:

* Add error handling
* Log each call
* Simulate actions in testing mode
* Enforce security limits (e.g., block dangerous tools)

---

## 🔐 Why Is Environment Important?

By abstracting execution into the `Environment`, you get:

| Benefit                   | Description                                                              |
| ------------------------- | ------------------------------------------------------------------------ |
| 🔒 Safety                 | You can prevent certain actions (e.g., deleting files in production)     |
| 🧪 Testability            | Swap in a “mock” environment to simulate behavior                        |
| ♻️ Reusability            | Use the same agent logic across local, cloud, or remote environments     |
| 📦 Separation of Concerns | The agent doesn't need to know *how* actions work — only that they exist |

---

## 👇 Where You've Seen It

In your agent code:

```python
result = self.environment.execute_action(action, invocation["args"])
```

This is where the agent is *handing off control* to the environment and saying:

> "I decided on this action — now go do it."





### 🤖 The `Environment` in GAME Is:

> **The execution layer where tools (Actions) are actually carried out.**

| Component             | Purpose                                                                |
| --------------------- | ---------------------------------------------------------------------- |
| 🧠 `Agent`            | The *brain* – plans, chooses actions based on goals and memory         |
| 🧰 `ActionRegistry`   | The *toolbelt* – contains tools (functions) the agent can use          |
| 📚 `Memory`           | The *diary* – stores previous thoughts, actions, and results           |
| 🧑‍💻 `AgentLanguage` | The *translator* – formats prompts and parses LLM replies              |
| 🌍 `Environment`      | The *hands + world* – **actually performs** the action chosen by agent |

---

### 🛠️ Example in Action

When the agent decides to call:

```json
{ "tool_name": "read_file", "args": { "filename": "report.txt" } }
```

**It does NOT run that tool itself.** Instead it says:

> “Hey `Environment`, I chose `read_file()` — please execute it with these args.”

The environment then:

* Looks up the corresponding function
* Calls `read_file(filename="report.txt")`
* Returns the result to the agent

---

### 🔁 Summary

The `Environment` is the **operational context** where tools run, files are accessed, and side effects happen. It's a **controlled layer** between the agent’s decisions and the real world — this gives you:

* **Control** (over what can and can’t happen)
* **Safety** (no rogue file deletions or uncontrolled APIs)
* **Swapability** (you can change how actions behave based on where the agent runs)
